In [ ]:
# | default_exp server

In [ ]:
# | export

import importlib
import sys
import asyncio
from typing import *
from contextlib import contextmanager
from pathlib import Path
import threading
import signal
from contextlib import ExitStack, contextmanager
from tempfile import TemporaryDirectory
import subprocess


import multiprocessing
from fastcore.meta import delegates
from fastcore.basics import patch
import typer
import asyncer

from fastkafka.application import FastKafka
from fastkafka.testing import change_dir
from fastkafka._components.helpers import _import_from_string, generate_app_src
from fastkafka._components.logger import get_logger

In [ ]:
import os
from time import sleep
from pydantic import BaseModel
from fastkafka._components.logger import supress_timestamps


In [ ]:
# | notest

# allows async calls in notebooks

import nest_asyncio

In [ ]:
# | notest

nest_asyncio.apply()

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
supress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
def create_test_app():
    app = FastKafka(bootstrap_servers="tvrtko-fastkafka-kafka-1:9092")

    class MyMessage(BaseModel):
        msg: str

    @app.consumes()
    def on_my_topic(msg: MyMessage):
        pass
    
    return app

In [ ]:
# | export

@contextmanager
def generate_app_in_tmp() -> Generator[str, None, None]:
    with TemporaryDirectory() as d:
        src_path = Path(d) / "main.py"
        generate_app_src(src_path)
        with change_dir(d):
            import_str = f"{src_path.stem}:kafka_app"
            yield import_str

In [ ]:
# | export


class ServerProcess:
    def __init__(self, app: str):
        self.app = app
        self.should_exit = False

    def run(self) -> None:
        return asyncio.run(self._serve())

    async def _serve(self) -> None:
        self._install_signal_handlers()

        self.application = _import_from_string(self.app)

        async with self.application:
            await self._main_loop()

    def _install_signal_handlers(self) -> None:
        if threading.current_thread() is not threading.main_thread():
            raise RuntimeError()

        loop = asyncio.get_event_loop()

        HANDLED_SIGNALS = (
            signal.SIGINT,  # Unix signal 2. Sent by Ctrl+C.
            signal.SIGTERM,  # Unix signal 15. Sent by `kill <pid>`.
        )

        def handle_exit(sig: int) -> None:
            self.should_exit = True

        for sig in HANDLED_SIGNALS:
            loop.add_signal_handler(sig, handle_exit, sig)

    async def _main_loop(self) -> None:
        while not self.should_exit:
            await asyncio.sleep(0.1)

In [ ]:
# | export

_app = typer.Typer()


@_app.command()
def run_fastkafka_server_process(
    app: str = typer.Argument(
        ...,
        help="input in the form of 'path:app', where **path** is the path to a python file and **app** is an object of type **FastKafka**.",
    )
) -> None:
    ServerProcess(app).run()

In [ ]:
# | notest

print("WARNING: make sure you save the notebook before running this cell\n")

print("Exporting and installing the new version of the CLI command...")
subprocess.run(["nbdev_export"], check=True, capture_output=True)
subprocess.run(["pip", "install", "-e", "..[dev]"], check=True, capture_output=True)

print("ok")


Exporting and installing the new version of the CLI command...
ok


In [ ]:
#| export


async def terminate_asyncio_process(p: asyncio.subprocess.Process) -> None:
    logger.info(f"terminate_asyncio_process(): Terminating the process {p.pid}...")
    # Check if SIGINT already propagated to process
    try:
        await asyncio.wait_for(p.wait(), 1)      
        logger.info(f"terminate_asyncio_process(): Process {p.pid} was already terminated.")
        return
    except asyncio.TimeoutError:
        pass
    
    for i in range(3):
        p.terminate()
        try:
            await asyncio.wait_for(p.wait(), 10)
            logger.info(f"terminate_asyncio_process(): Process {p.pid} terminated.")
            return
        except asyncio.TimeoutError:
            logger.warning(f"terminate_asyncio_process(): Process {p.pid} not terminated, retrying...")

    logger.warning(f"Killing the process {p.pid}...")
    p.kill()
    p.wait()
    logger.warning(f"terminate_asyncio_process(): Process {p.pid} killed!")

In [ ]:
with generate_app_in_tmp() as app:
    proc = await asyncio.create_subprocess_exec(
                "run_fastkafka_server_process", app, stdout=asyncio.subprocess.PIPE
            )
    sleep(5)
    await terminate_asyncio_process(proc)
    outputs, _ = await proc.communicate()
    
    print(outputs.decode("utf-8"))
    assert proc.returncode == 0

[INFO] __main__: terminate_asyncio_process(): Terminating the process 640...
[INFO] __main__: terminate_asyncio_process(): Process 640 terminated.
[INFO] main: check
[INFO] fastkafka._components.asyncapi: Old async specifications at '/tmp/tmpe3d2rcyp/asyncapi/spec/asyncapi.yml' does not exist.
[INFO] fastkafka._components.asyncapi: New async specifications generated at: '/tmp/tmpe3d2rcyp/asyncapi/spec/asyncapi.yml'
[INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092'}'
[INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092'}'
[INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092'}'
[INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092'}'
[INFO] fa

In [ ]:
# | export

async def run_fastkafka_server(num_workers: int, app: str) -> None:
    loop = asyncio.get_event_loop()

    HANDLED_SIGNALS = (
        signal.SIGINT,  # Unix signal 2. Sent by Ctrl+C.
        signal.SIGTERM,  # Unix signal 15. Sent by `kill <pid>`.
    )

    d = {"should_exit": False}

    def handle_exit(sig: int, d: Dict[str, bool] = d) -> None:
        d["should_exit"] = True

    for sig in HANDLED_SIGNALS:
        loop.add_signal_handler(sig, handle_exit, sig)

    async with asyncer.create_task_group() as tg:
        tasks = [
            tg.soonify(asyncio.create_subprocess_exec)(
                "run_fastkafka_server_process", app, stdout=asyncio.subprocess.PIPE, stdin=asyncio.subprocess.PIPE
            )
            for i in range(num_workers)
        ]
        
    procs = [task.value for task in tasks]

    async def log_output(output: Optional[asyncio.StreamReader], pid: int, d: Dict[str, bool] = d) -> None:
        if output is None:
            raise RuntimeError("Expected StreamReader, got None. Is stdout piped?")
        while not output.at_eof():
            outs = await output.readline()
            if outs != b"":
                typer.echo(f"[{pid:03d}]: " + outs.decode("utf-8"), nl=False)
    
    async with asyncer.create_task_group() as tg:
        for proc in procs:
            tg.soonify(log_output)(proc.stdout, proc.pid)
            
        while not d["should_exit"]:
            await asyncio.sleep(0.2)
            
        typer.echo("Starting process cleanup, this may take a few seconds...")
        for proc in procs:
            tg.soonify(terminate_asyncio_process)(proc)
        
    for proc in procs:
        output, _ = await proc.communicate()
        if output:
            typer.echo(f"[{proc.pid:03d}]: " + output.decode("utf-8"), nl=False)

    returncodes = [proc.returncode for proc in procs]
    if not returncodes == [0] * len(procs):
        typer.secho(
            f"Return codes are not all zero: {returncodes}",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(1)

In [ ]:
# | notest

with generate_app_in_tmp() as app:
    print(f"{app=}")
    await run_fastkafka_server(4, app)

print("ok")

app='main:kafka_app'
[651]: [INFO] main: check
[651]: [INFO] fastkafka._components.asyncapi: Old async specifications at '/tmp/tmpf4dlembg/asyncapi/spec/asyncapi.yml' does not exist.
[651]: [INFO] fastkafka._components.asyncapi: New async specifications generated at: '/tmp/tmpf4dlembg/asyncapi/spec/asyncapi.yml'
[651]: [INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092'}'
[651]: [INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092'}'
[651]: [INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092'}'
[651]: [INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092'}'
[651]: [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() s

[647]: [INFO] aiokafka.consumer.group_coordinator: (Re-)joining group tvrtko-fastkafka-kafka-1:9092_group
[645]: [INFO] fastkafka._components.asyncapi: Keeping the old async specifications at: '/tmp/tmpf4dlembg/asyncapi/spec/asyncapi.yml'
[645]: [INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092'}'
[647]: [INFO] aiokafka.consumer.group_coordinator: Discovered coordinator 1001 for group tvrtko-fastkafka-kafka-1:9092_group
[647]: [INFO] aiokafka.consumer.group_coordinator: Revoking previously assigned partitions set() for group tvrtko-fastkafka-kafka-1:9092_group
[647]: [INFO] aiokafka.consumer.group_coordinator: (Re-)joining group tvrtko-fastkafka-kafka-1:9092_group
[649]: [INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092'}'
[645]: [INFO] fastkafka.application: _create_producer() : created producer using the co

[651]: [INFO] aiokafka.consumer.group_coordinator: Elected group leader -- performing partition assignments using roundrobin
[645]: [INFO] aiokafka.consumer.group_coordinator: Joined group 'tvrtko-fastkafka-kafka-1:9092_group' (generation 70) with member_id aiokafka-0.8.0-761d6f45-dfe6-411b-8a9e-444e80151346
[649]: [INFO] aiokafka.consumer.group_coordinator: Successfully synced group tvrtko-fastkafka-kafka-1:9092_group with generation 70
[649]: [INFO] aiokafka.consumer.group_coordinator: Setting newly assigned partitions set() for group tvrtko-fastkafka-kafka-1:9092_group
[649]: [INFO] aiokafka.consumer.group_coordinator: Successfully synced group tvrtko-fastkafka-kafka-1:9092_group with generation 70
[649]: [INFO] aiokafka.consumer.group_coordinator: Setting newly assigned partitions set() for group tvrtko-fastkafka-kafka-1:9092_group
[645]: [INFO] aiokafka.consumer.group_coordinator: Successfully synced group tvrtko-fastkafka-kafka-1:9092_group with generation 70
[645]: [INFO] aiokaf